In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.config import make_config
from deeprte.model.tf.input_pipeline import shard
config = make_config()

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config

best_model_eval_metric: ''
best_model_eval_metric_higher_is_better: true
checkpoint_interval_type: null
dataset:
  buffer_size: 5000
  data_split:
    is_split_datasets: false
    num_samples: 10
    num_test_samples: 2
    train_validation_split_rate: 0.8
  max_intra_op_parallelism: 1
  seed: 42
  threadpool_size: 48
  train:
    batch_size: 6
    collocation_sizes: 500
    repeat: 1
  validation:
    batch_size: 2
eval_initial_weights: false
experiment_kwargs: {}
interval_type: secs
log_all_train_data: false
log_tensors_interval: 60
log_train_data_interval: 120.0
logging_interval_type: null
max_checkpoints_to_keep: 5
one_off_evaluate: false
random_mode_eval: same_host_same_device
random_mode_train: unique_host_unique_device
random_seed: 42
save_checkpoint_interval: 300
train_checkpoint_all_hosts: false
training_steps: 10000

In [4]:
data_path = "/workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat"

from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline()
data = data_pipeline.process(data_path)

from deeprte.model.tf.rte_dataset import np_to_tensor_dict

tf_data = np_to_tensor_dict(data)

from deeprte.model.tf.rte_dataset import divide_batch_feat
batched_data, unbatched_data = divide_batch_feat(tf_data)

In [5]:
import tensorflow as tf

tf.nest.map_structure(lambda x:x.shape, tf_data)

{'sigma': TensorShape([10, 1681, 2]),
 'psi_label': TensorShape([10, 40344]),
 'boundary': TensorShape([10, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'scattering_kernel': TensorShape([10, 40344, 24]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [8]:
data_config = config.dataset

In [27]:
from deeprte.model.tf.input_pipeline import load_and_split_data,process_features,np_example_to_features

ds, unbatched_feat = load_and_split_data(tf_data, data_config, True)

In [32]:
data_config.data_split.is_split_datasets = False

In [33]:
ds_without_split, unbatched_feat = load_and_split_data(
        features=tf_data,
        config=data_config,
        pre_shuffle=True,
        seed=data_config.seed,
    )

In [34]:
tfds_without_split = process_features(
            ds=ds_without_split,
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=[2,5],
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
        )

In [35]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_without_split))

{'sigma': (2, 5, 1681, 2),
 'psi_label': (2, 5, 40344),
 'boundary': (2, 5, 1968),
 'scattering_kernel': (2, 5, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [37]:
data_config.data_split.is_split_datasets = True
data_config

buffer_size: 5000
data_split:
  is_split_datasets: true
  num_samples: 10
  num_test_samples: 2
  train_validation_split_rate: 0.8
max_intra_op_parallelism: 1
seed: 42
threadpool_size: 48
train:
  batch_size: 6
  collocation_sizes: 500
  repeat: 1
validation:
  batch_size: 2

In [38]:
ds_with_split, unbatched_feat = load_and_split_data(
        features=tf_data,
        config=data_config,
        pre_shuffle=True,
        seed=data_config.seed,
    )

In [39]:
len(ds_with_split)

3

In [40]:
train_ds = process_features(
            ds=ds_with_split[1],
            unbatched_feat=unbatched_feat,
            is_training=True,
            batch_sizes=[2,3],
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [41]:
tf.nest.map_structure(lambda x:x.shape, next(train_ds))

{'sigma': (2, 3, 1681, 2),
 'psi_label': (2, 3, 500),
 'boundary': (2, 3, 1968),
 'scattering_kernel': (2, 3, 500, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 500, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [46]:
val_ds = process_features(
            ds=ds_with_split[2],
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=[2,1],
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [47]:
tf.nest.map_structure(lambda x:x.shape, next(val_ds))

{'sigma': (2, 1, 1681, 2),
 'psi_label': (2, 1, 40344),
 'boundary': (2, 1, 1968),
 'scattering_kernel': (2, 1, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [51]:
test_ds = process_features(
            ds=ds_with_split[0],
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=[2,1],
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [52]:
tf.nest.map_structure(lambda x:x.shape, next(test_ds))

{'sigma': (2, 1, 1681, 2),
 'psi_label': (2, 1, 40344),
 'boundary': (2, 1, 1968),
 'scattering_kernel': (2, 1, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}